In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv("temp_data.csv")

In [3]:
df.head(20)

,SentAt,StartedAt,Control,Sad,Intrusion,Encourage,FunThings,IgotThis,Company,Alone,...,Out,Eaten,Avoid,Useful,Enjoy,Pleas,Pleasant,Unpl,Unpleasant,Remarks
0,15/03/2017 16:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/03/2017 09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16/03/2017 15:12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16/03/2017 15:30,16/03/2017 15:30,70.00,42.02,76.74,59.38,35.54,81.14,0.0,24.92,...,66.63,61.97,20.52,41.50,79.85,1.0,50.83,1.0,46.68,NaN
4,16/03/2017 21:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,17/03/2017 09:00,17/03/2017 09:17,17.53,18.74,16.93,81.39,82.59,82.19,1.0,16.93,...,87.41,69.84,2.17,87.11,77.77,1.0,88.92,0.0,NaN,NaN
6,17/03/2017 15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,17/03/2017 21:00,17/03/2017 21:05,26.87,27.07,30.18,69.34,54.88,45.44,1.0,19.64,...,62.31,31.39,2.17,50.96,54.48,1.0,73.56,1.0,71.65,NaN
8,18/03/2017 09:00,18/03/2017 09:11,4.78,2.27,10.00,88.62,77.77,79.28,1.0,0.00,...,1.97,51.16,2.87,22.05,70.84,1.0,65.52,0.0,NaN,NaN
9,18/03/2017 15:00,18/03/2017 15:07,1.46,0.00,2.47,82.89,82.69,82.79,1.0,3.07,...,84.00,82.69,1.27,59.60,84.30,1.0,88.31,0.0,NaN,NaN


In [4]:
print(df.shape)
print(df.isna().sum())
print(df.columns)

(1235, 27)
SentAt             0
StartedAt        691
Control          691
Sad              691
Intrusion        691
Encourage        691
FunThings        691
IgotThis         691
Company          691
Alone            691
Thoughts         691
Safe             691
Appointments     691
Cancel           691
Laying           691
Sleep            691
Qualitysleep    1072
Out              691
Eaten            691
Avoid            691
Useful           691
Enjoy            691
Pleas            691
Pleasant        1076
Unpl             691
Unpleasant      1211
Remarks         1087
dtype: int64
Index(['SentAt', 'StartedAt', 'Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Qualitysleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Pleasant', 'Unpl',
       'Unpleasant', 'Remarks'],
      dtype='object')


In [5]:
df = df.drop(columns = ['StartedAt', 'Qualitysleep', 'Pleasant', 'Unpleasant', 'Remarks'])

In [6]:
#drop all observations hich have all missing except for SentAt
to_drop =  ['Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Unpl']

df = df.dropna(subset = to_drop, how = 'any', axis=0)

In [7]:
#make idvar
df['id_var'] = 1
#make date
df['Date'] = pd.to_datetime(df['SentAt'], format = '%d/%m/%Y %H:%M')
#get the date and hour
df['Date_2'] = [d.strftime("%x") for d in df['Date']]
df['Hour'] = [d.strftime("%H") for d in df['Date']]
#make the beepvar required for graphicalvar
df['first_obs'] = [1 if (x == '09' or x == '10') else 0 for x in df['Hour']]
df['second_obs'] = [2 if (x == '15' or x == '16') else 0 for x in df['Hour']]
df['third_obs'] = [3 if (x == '21' or x == '22') else 0 for x in df['Hour']]
df['beep_var'] = df['first_obs'] + df['second_obs'] + df['third_obs']
#order by date
df = df.sort_values(by='Date')

In [8]:
#make day_var
day_var = pd.DataFrame(pd.date_range(df['Date'].min(), df['Date'].max()))
day_var.columns = ['Date_Range']
day_var['day_var'] = [d.strftime("%x") for d in day_var['Date_Range']]
day_var = day_var.drop('Date_Range', axis = 1).reset_index()
df = df.merge(day_var, how = 'left', left_on = 'Date_2', right_on = 'day_var')

In [9]:
#clean by dropping unnecessary columns and renaming columns
df = df.drop(['first_obs', 'second_obs', 'third_obs', 'day_var'], axis = 1)
#rename index to day_var
df = df.rename(columns = {'index': 'day_var'})
#because R doesn't start from -
df['day_var'] = df['day_var']  + 1

In [15]:
df['consecutive'] = df['day_var'].diff()

In [16]:
df['consecutive'].value_counts()

1.0    271
0.0    215
2.0     43
3.0      9
4.0      3
6.0      1
5.0      1
Name: consecutive, dtype: int64

In [14]:
check = df['consecutive'] > 1
check.sum()

57

In [17]:
df[df['consecutive'] > 1]

,SentAt,Control,Sad,Intrusion,Encourage,FunThings,IgotThis,Company,Alone,Thoughts,...,Enjoy,Pleas,Unpl,id_var,Date,Date_2,Hour,beep_var,day_var,consecutive
26,28/03/2017 09:00,17.73,19.04,30.88,70.54,57.09,40.02,0.0,13.22,10.10,...,56.69,0.0,0.0,1,2017-03-28 09:00:00,03/28/17,09,1,13,2.0
139,19/05/2017 15:00,16.01,14.87,12.13,87.55,53.60,70.69,0.0,25.35,10.31,...,73.36,1.0,0.0,1,2017-05-19 15:00:00,05/19/17,15,2,65,2.0
141,26/05/2017 15:00,8.76,18.85,3.87,81.54,62.89,69.61,0.0,8.15,6.31,...,72.67,0.0,0.0,1,2017-05-26 15:00:00,05/26/17,15,2,72,6.0
143,29/05/2017 15:00,11.21,12.43,12.73,69.31,57.99,60.44,0.0,19.16,10.90,...,39.34,0.0,0.0,1,2017-05-29 15:00:00,05/29/17,15,2,75,3.0
172,13/06/2017 09:00,5.70,5.39,18.24,69.00,71.45,66.56,1.0,15.49,4.78,...,79.40,0.0,0.0,1,2017-06-13 09:00:00,06/13/17,09,1,90,2.0
211,04/07/2017 09:00,29.86,11.51,9.98,74.51,50.04,50.65,0.0,17.63,9.37,...,68.09,0.0,0.0,1,2017-07-04 09:00:00,07/04/17,09,1,111,2.0
219,10/07/2017 15:00,46.68,43.01,47.29,64.73,19.46,35.98,0.0,51.26,55.55,...,13.35,0.0,0.0,1,2017-07-10 15:00:00,07/10/17,15,2,117,2.0
225,15/07/2017 21:00,13.65,16.40,13.35,68.39,70.84,56.77,0.0,15.49,15.18,...,51.57,0.0,0.0,1,2017-07-15 21:00:00,07/15/17,21,3,122,3.0
231,21/07/2017 15:00,13.96,11.21,13.96,71.45,55.85,46.68,0.0,12.12,8.45,...,32.92,0.0,0.0,1,2017-07-21 15:00:00,07/21/17,15,2,128,2.0
232,24/07/2017 09:00,9.98,21.90,12.43,63.80,25.88,43.01,1.0,16.40,3.87,...,33.84,0.0,0.0,1,2017-07-24 09:00:00,07/24/17,09,1,131,3.0


In [19]:
df['day_var'].head(50)

0      1
1      2
2      2
3      3
4      3
5      3
6      4
7      4
8      5
9      5
10     5
11     6
12     6
13     7
14     7
15     8
16     8
17     9
18     9
19     9
20    10
21    10
22    10
23    11
24    11
25    11
26    13
27    13
28    13
29    14
30    14
31    14
32    15
33    15
34    15
35    16
36    16
37    17
38    17
39    18
40    18
41    19
42    19
43    19
44    20
45    20
46    20
47    21
48    21
49    22
Name: day_var, dtype: int64

In [ ]:
print(df.shape)
print(df.isna().sum().sum())
print(df.columns)

In [ ]:

#first = 16/03/2017
#last = 27/04/2018
# just over a year in scope
# note that we have 544 obs but not evenly spaces out

In [ ]:
df.head()